In [1]:
import os

In [2]:
import random

In [3]:
from os.path import join

In [4]:
data_prefix = "/home/evan/data/climate"

In [5]:
source_file_prefix = "/home/evan/data/climate/lists/small-gpu-size_lists/supervised/"

In [6]:
names = ["climo"]#["2xCO2", "SST_plus2",  "climo"]

In [9]:
num_crops = 1
im_listname = "image_files.txt"
lbl_listname = "label_files.txt"
crop_listname = "crop_indices.txt"
im_txt_file = join(source_file_prefix,im_listname)
lbl_txt_file = join(source_file_prefix,lbl_listname)
crop_ind_txt_file = join(source_file_prefix,crop_listname)
for name in names:
    path = join(data_prefix,name)
    imdir = join(path,"images")
    ims = os.listdir(imdir)
    lbldir = join(path,"labels")
    lbls = os.listdir(lbldir)
    ims.sort()
    lbls.sort()
    random.seed(3)
    random.shuffle(ims)
    random.seed(3)
    random.shuffle(lbls)
    
    
    num_ex = len(ims)
    num_tr = int(0.6*num_ex)
    num_val = int(0.2*num_ex)
    for crop_ind in range(1):
        write_train_val_test(ims,imdir,im_listname,source_file_prefix)
        write_train_val_test(lbls,lbldir,lbl_listname,source_file_prefix)
#         for pref in ["tr_", "val_", "test_"]:
#             with open(join(source_file_prefix,pref + crop_listname), "a") as f:
#                 for _ in lbls:
#                     f.write(str(crop_ind) + "\n")

In [8]:
def write_train_val_test(datanames,datadir, txt_file, sourcedir):
    num_ex = len(datanames)
    num_tr = int(0.6*num_ex)
    num_val = int(0.2*num_ex)
    
    fname = os.path.basename(txt_file)
    dirname = os.path.dirname(txt_file)
    with open(join(sourcedir, "tr_"+ txt_file),"a") as f:
        for dataname in datanames[:num_tr]:
            f.writelines(join(datadir,dataname) + "\n")
    
    with open(join(sourcedir, "val_"+ txt_file),"a") as f:
        for dataname in datanames[num_tr:num_tr+num_val]:
            f.writelines(join(datadir,dataname) + "\n")
    
    with open(join(sourcedir, "test_"+ txt_file),"a") as f:
        for dataname in datanames[num_tr+num_val:]:
            f.writelines(join(datadir,dataname) + "\n")
    

In [8]:
# #shuffle everything the same
# for txt_file in [im_txt_file, lbl_txt_file, crop_ind_txt_file]:
#     with open(txt_file, "r") as f:
#         random.seed(3)
#         lines = f.readlines()
#         random.shuffle(lines)
#     with open(txt_file, "w") as f:
#         f.writelines(lines)
            
    

# # split into train val test

# for txt_file in [im_txt_file, lbl_txt_file, crop_ind_txt_file]:
#     with open(txt_file, "r") as f:
#         lines = f.readlines()
#         num_ex = len(lines)
#         num_tr = int(0.6*num_ex)
#         num_val = int(0.2*num_ex)
    
#     fname = os.path.basename(txt_file)
#     dirname = os.path.dirname(txt_file)
#     with open(join(dirname, "tr_"+ fname),"w") as f:
#         f.writelines(lines[:num_tr])
    
#     with open(join(dirname, "val_"+ fname), "w") as f:
#         f.writelines(lines[num_tr:num_tr+num_val])
    
#     with open(join(dirname, "test_"+ fname), "w") as f:
#         f.writelines(lines[num_tr+num_val:])
    
    
    